<a href="https://colab.research.google.com/github/Bhakthi-Shetty7811/hiver-assignment/blob/main/Part_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part A: Email Tagging Mini-System

**Candidate:** Bhakthi Shetty

This notebook implements a baseline email tagging system for customer support emails.  

**Approach:**
- Rule-based guardrails for common patterns and anti-patterns
- Simulated LLM predictions (placeholder for real API)
- Fallback recovery for cases where rules and LLM fail

The system ensures **customer isolation**, meaning tags from one customer are never applied to another.

**Step 1: Import Libraries**

In [71]:
import pandas as pd

# For JSON output (simulating LLM output)
import json

**Step 2: Load Dataset**

In [72]:
!rm *.csv

In [73]:
from google.colab import files
uploaded = files.upload()

import pandas as pd

# Load based on whatever names were uploaded
small_df = pd.read_csv([f for f in uploaded.keys() if "small" in f.lower()][0])
large_df = pd.read_csv([f for f in uploaded.keys() if "large" in f.lower()][0])

small_df["text"] = small_df["subject"] + " " + small_df["body"]
large_df["text"] = large_df["subject"] + " " + large_df["body"]

small_df.head(), large_df.head()

Saving large_dataset.csv to large_dataset.csv
Saving small_dataset.csv to small_dataset.csv


(   email_id customer_id                              subject  \
 0         1      CUST_A      Unable to access shared mailbox   
 1         2      CUST_A                    Rules not working   
 2         3      CUST_A               Email stuck in pending   
 3         4      CUST_B  Automation creating duplicate tasks   
 4         5      CUST_B                         Tags missing   
 
                                                 body             tag  \
 0  Hi team, I'm unable to access the shared mailb...    access_issue   
 1  We created a rule to auto-assign emails based ...  workflow_issue   
 2  One of our emails is stuck in pending even aft...      status_bug   
 3  Your automation engine is creating 2 tasks for...  automation_bug   
 4  Many of our tags are not appearing for new ema...   tagging_issue   
 
                                                 text  
 0  Unable to access shared mailbox Hi team, I'm u...  
 1  Rules not working We created a rule to auto-as...  


**Step 3: Create customer-level tag mapping**

Before adding customer isolation, the model mixed tags across customers and accuracy was ~42%.

Restricting allowed tags per customer raised accuracy to ~91%.

In [74]:
customer_tags = {}
for customer_id, group in small_df.groupby("customer_id"):
    customer_tags[customer_id] = list(group["tag"].unique())

customer_tags

{'CUST_A': ['access_issue', 'workflow_issue', 'status_bug'],
 'CUST_B': ['automation_bug', 'tagging_issue', 'billing'],
 'CUST_C': ['analytics_issue', 'performance', 'setup_help'],
 'CUST_D': ['mail_merge_issue', 'user_management', 'feature_request']}

**Step 4: Rule-based Guardrails**

The rules prioritize known strong signals before calling LLM

In [75]:
def apply_rules(text, allowed_tags):
    text_lower = text.lower()

    # --- 0. Feature Request (TOP PRIORITY) ---
    if "feature request" in text_lower or text_lower.startswith("feature request"):
        if "feature_request" in allowed_tags:
            return "feature_request"

    # --- 1. Access / Login ---
    if any(k in text_lower for k in ["access", "permission", "denied", "auth", "authorization"]):
        for t in ["access_issue", "auth_issue"]:
            if t in allowed_tags:
                return t

    # --- 2. Workflow / Rules ---
    if any(k in text_lower for k in ["rule", "workflow", "auto-assign", "trigger"]):
        for t in ["workflow_issue", "workflow_bug", "automation_issue", "automation_bug"]:
            if t in allowed_tags:
                return t

    # --- 3. Pending / Stuck / Queue ---
    if any(k in text_lower for k in ["pending", "stuck", "queued", "not moving"]):
        for t in ["status_bug", "sync_delay", "automation_delay"]:
            if t in allowed_tags:
                return t

    # --- 4. Duplicate ---
    if "duplicate" in text_lower:
        for t in ["duplication_bug", "automation_bug"]:
            if t in allowed_tags:
                return t

    # --- 5. Mail Merge ---
    if "mail merge" in text_lower:
        if "mail_merge_issue" in allowed_tags:
            return "mail_merge_issue"

    # --- 6. Billing ---
    if any(k in text_lower for k in ["billing", "invoice", "charged incorrectly"]):
        for t in ["billing", "billing_error"]:
            if t in allowed_tags:
                return t

    # --- 7. CSAT / Analytics ---
    if "csat" in text_lower:
        # visibility issues go to analytics
        if any(k in text_lower for k in ["not visible", "disappear", "missing"]):
            if "analytics_issue" in allowed_tags:
                return "analytics_issue"
        for t in ["csat_issue", "analytics_issue"]:
            if t in allowed_tags:
                return t

    if any(k in text_lower for k in ["analytics", "dashboard", "report"]):
        for t in ["analytics_issue", "analytics_accuracy"]:
            if t in allowed_tags:
                return t

    # --- 8. Performance ---
    if any(k in text_lower for k in ["slow", "delay", "loading"]):
        for t in ["performance", "ui_performance", "analytics_latency"]:
            if t in allowed_tags:
                return t

    # --- 9. Setup / SLA ---
    if "setup" in text_lower or "set up" in text_lower:
        if "setup_help" in allowed_tags:
            return "setup_help"

    if "sla" in text_lower:
        if "sla_issue" in allowed_tags:
            return "sla_issue"

    # --- 10. Tagging ---
    if "tag" in text_lower:
        for t in ["tagging_issue", "tagging_accuracy"]:
            if t in allowed_tags:
                return t

    # --- 11. User Management ---
    if "user" in text_lower and any(k in text_lower for k in ["add", "authorization"]):
        if "user_management" in allowed_tags:
            return "user_management"

    return None



**Step 5: LLM Placeholder**

Currently disabled since I’m building a baseline without API usage

In [76]:
def llm_predict(email_text, allowed_tags):
    """
    Simulated LLM prediction function.
    Replace with OpenAI call if API is available.
    """
    # Example prompt construction (not run)
    prompt = f"""
You are a classifier for customer support emails.
Assign exactly one tag from the allowed list below.

EMAIL:
{email_text}

ALLOWED TAGS FOR THIS CUSTOMER:
{allowed_tags}

OUTPUT FORMAT (JSON):
{{"predicted_tag": "<tag>"}}
"""

    # Example: If API available, uncomment
    """
    import openai
    openai.api_key = "YOUR_API_KEY"

    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        temperature=0,
        max_tokens=50
    )
    predicted_tag = response.choices[0].text.strip()
    # Convert JSON to dictionary
    predicted_tag = json.loads(predicted_tag)["predicted_tag"]
    return predicted_tag
    """

    # Since key is expired / free limit exceeded, return None for demonstration
    return None

**Step 6 : Fallback if rules and LLM (None) both fail**



In [77]:
# Picks the closest tag by matching keyword groups

def smart_fallback(text, allowed_tags):
    text_lower = text.lower()

    fallback_groups = {
        "merge": ["mail_merge_issue"],
        "duplicate": ["duplication_bug", "automation_bug"],
        "slow": ["performance", "ui_performance", "analytics_latency"],
        "delay": ["performance", "analytics_latency"],
        "tag": ["tagging_issue", "tagging_accuracy"],
        "setup": ["setup_help"],
        "user": ["user_management"],
        "access": ["access_issue"],
        "billing": ["billing_error"],
        "analytics": ["analytics_issue"]
    }

    # Match by keyword grouping
    for keyword, taglist in fallback_groups.items():
        if keyword in text_lower:
            for t in taglist:
                if t in allowed_tags:
                    return t

    # If still no match → pick most common / safe tag
    safe_priority = ["feature_request", "analytics_issue", "ui_bug", "workflow_issue"]

    for t in safe_priority:
        if t in allowed_tags:
            return t

    # Last fallback → first allowed tag
    return allowed_tags[0]


**Step 7: Prediction + Evaluation**

Pipeline: Rules → LLM (optional) → Smart fallback

In [78]:
predicted_tags = []

for i, row in small_df.iterrows():
    customer_id = row["customer_id"]
    allowed_tags = customer_tags[customer_id]
    text = row["text"]

    # 1. RULE ENGINE
    tag = apply_rules(text, allowed_tags)

    # 2. IF RULE FAILS → LLM
    if not tag:
        tag = llm_predict(text, allowed_tags)

    # 3. IF LLM FAILS → SMART FALLBACK
    if not tag:
        tag = smart_fallback(text, allowed_tags)

    # 4. GUARANTEE TAG
    if not tag:
        tag = allowed_tags[0]

    predicted_tags.append(tag)

small_df["predicted_tag"] = predicted_tags


print("Prediction Results:")
print(small_df[["email_id","customer_id","text","tag","predicted_tag"]])


Prediction Results:
    email_id customer_id                                               text  \
0          1      CUST_A  Unable to access shared mailbox Hi team, I'm u...   
1          2      CUST_A  Rules not working We created a rule to auto-as...   
2          3      CUST_A  Email stuck in pending One of our emails is st...   
3          4      CUST_B  Automation creating duplicate tasks Your autom...   
4          5      CUST_B  Tags missing Many of our tags are not appearin...   
5          6      CUST_B  Billing query We were charged incorrectly this...   
6          7      CUST_C  CSAT not visible CSAT scores disappeared from ...   
7          8      CUST_C  Delay in email loading Opening a conversation ...   
8          9      CUST_C  Need help setting up SLAs We want to configure...   
9         10      CUST_D  Mail merge failing Mail merge is not sending e...   
10        11      CUST_D  Can't add new user Trying to add a new team me...   
11        12      CUST_D  Featur

**Step 8: Accuracy check + Error Review**

Helps identify misclassifications for rule tuning.

In [80]:
correct = 0
mistakes = []

for i, row in small_df.iterrows():
    true_tag = row["tag"]
    predicted = row["predicted_tag"]

    if predicted == true_tag:
        correct += 1
    else:
        mistakes.append((row["email_id"], true_tag, predicted, row["text"][:80]))

total = len(small_df)
accuracy = (correct / total) * 100

print(f"\nTOTAL EMAILS: {total}")
print(f"CORRECT PREDICTIONS: {correct}")
print(f"ACCURACY: {accuracy:.2f}%")

print("\n--- MISTAKES ---")
for m in mistakes:
    print(f"Email {m[0]} | TRUE: {m[1]} | PRED: {m[2]}")
    print(f"Snippet: {m[3]}")
    print("------")



TOTAL EMAILS: 12
CORRECT PREDICTIONS: 11
ACCURACY: 91.67%

--- MISTAKES ---
Email 9 | TRUE: setup_help | PRED: analytics_issue
Snippet: Need help setting up SLAs We want to configure SLAs for different customer tiers
------


In [81]:
print("\n=== Evaluation Completed ===")



=== Evaluation Completed ===
